In [25]:
import os

directory = 'data'

texts = []
labels = []

for filename in os.listdir(directory):
    if filename.endswith('.txt'):  
        with open(os.path.join(directory, filename), 'r', encoding='utf-8') as file:
            for line in file:
                if '|' in line:
                    parts = line.rsplit('|', 1)
                    if parts[1].strip() == "":
                        continue
                    text = parts[0].strip()
                    label = parts[1].strip() if len(parts) > 1 else 'O'  
                    texts.append(text)
                    labels.append(label)

In [26]:
from collections import Counter
Counter(labels)

Counter({'O': 10209,
         'AttackRansom': 1494,
         'AttackRansom, AttackDatabreach': 24,
         'DiscoverVulnerability': 1467,
         'AttackDatabreach': 1405,
         'PatchVulnerability': 949,
         'AttackPhishing': 1283,
         'PatchVulnerability, DiscoverVulnerability': 83,
         'DiscoverVulnerability , PatchVulnerability': 40,
         'DiscoverVulnerability, PatchVulnerability': 121,
         'AttackPhishing, AttackDatabreach': 26,
         'AttackDatabreach, AttackPhishing': 21,
         'DiscoverVulnerability, AttackDatabreach': 22,
         'PatchVulnerability, AttackRansom': 4,
         'PatchVulnerability , DiscoverVulnerability': 9,
         'DiscoverVulnerability, AttackPhishing': 2,
         'AttackRansom, AttackDatabreach, AttackPhishing': 1,
         'AttackDatabreach, AttackRansom': 14,
         'PatchVulnerability , AttackDatabreach': 1,
         'AttackDatabreach, DiscoverVulnerability': 8,
         'AttackDatabreach, PatchVulnerability': 3,

In [27]:
temp  = []
for label in labels:
    for t in label.split(','):
        temp.append(t.strip())

In [28]:
Counter(temp)

Counter({'O': 10209,
         'AttackRansom': 1581,
         'AttackDatabreach': 1571,
         'DiscoverVulnerability': 1771,
         'PatchVulnerability': 1228,
         'AttackPhishing': 1374})